In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine, text
import dotenv
dotenv.load_dotenv('../config/.env')

True

In [2]:
PATH = '../datasets/dados_clinicos/dados_clinicos.csv'
df_dados_clinicos = pd.read_csv(PATH, sep=';', encoding='latin1')

In [3]:
df_dados_clinicos.head()

,id_cliente,peso,colesterol,genero
0,1,152.7,203.9,Masculino
1,2,184.8,233.8,Masculino
2,3,152.1,210.4,Masculino
3,4,139.8,196.8,Masculino
4,5,182.4,215.8,Feminino


In [4]:
PATH = '../datasets/dados_estado_regiao/estado_regiao.csv'
df_estado_regiao = pd.read_csv(PATH, sep=';', encoding='latin1')

In [5]:
df_estado_regiao.head()

,id_estado,sigla,estado,regiao,pais
0,1,AC,Acre,Norte,Brasil
1,2,AL,Alagoas,Nordeste,Brasil
2,3,AP,Amapá,Norte,Brasil
3,4,AM,Amazonas,Norte,Brasil
4,5,BA,Bahia,Nordeste,Brasil


In [6]:
PATH = '../datasets/dados_pacientes/dados_pacientes.csv'
df_dados_pacientes = pd.read_csv(PATH, sep=';', encoding='latin1')

In [7]:
df_dados_pacientes.head()

,id_cliente,idade,classe_trabalho,escolaridade,id_estado,estado_civil,raca,qtde_filhos,salario
0,1,39,Servidor Público,Ensino Médio Completo,2,Solteiro,Branco,2.0,4754
1,2,50,Autônomo,Superior Incompleto,24,Casado,Branco,1.0,3923
2,3,38,Funcionário Setor Privado,Ensino Médio Incompleto,4,Divorciado,Branco,0.0,1100
3,4,53,Funcionário Setor Privado,Ensino Médio Incompleto,24,Casado,Negro,1.0,1100
4,5,28,Funcionário Setor Privado,Ensino Médio Completo,15,Casado,Negro,0.0,3430


In [8]:
df = pd.merge(left=df_dados_clinicos, right=df_dados_pacientes, left_on='id_cliente', right_on='id_cliente', how='inner')

In [9]:
df.head()

,id_cliente,peso,colesterol,genero,idade,classe_trabalho,escolaridade,id_estado,estado_civil,raca,qtde_filhos,salario
0,1,152.7,203.9,Masculino,39,Servidor Público,Ensino Médio Completo,2,Solteiro,Branco,2.0,4754
1,2,184.8,233.8,Masculino,50,Autônomo,Superior Incompleto,24,Casado,Branco,1.0,3923
2,3,152.1,210.4,Masculino,38,Funcionário Setor Privado,Ensino Médio Incompleto,4,Divorciado,Branco,0.0,1100
3,4,139.8,196.8,Masculino,53,Funcionário Setor Privado,Ensino Médio Incompleto,24,Casado,Negro,1.0,1100
4,5,182.4,215.8,Feminino,28,Funcionário Setor Privado,Ensino Médio Completo,15,Casado,Negro,0.0,3430


In [10]:
df = pd.merge(left=df_dados_pacientes, right=df_estado_regiao, left_on='id_estado', right_on='id_estado', how='inner')

In [11]:
df.head()

,id_cliente,idade,classe_trabalho,escolaridade,id_estado,estado_civil,raca,qtde_filhos,salario,sigla,estado,regiao,pais
0,1,39,Servidor Público,Ensino Médio Completo,2,Solteiro,Branco,2.0,4754,AL,Alagoas,Nordeste,Brasil
1,2,50,Autônomo,Superior Incompleto,24,Casado,Branco,1.0,3923,SC,Santa Catarina,Sul,Brasil
2,3,38,Funcionário Setor Privado,Ensino Médio Incompleto,4,Divorciado,Branco,0.0,1100,AM,Amazonas,Norte,Brasil
3,4,53,Funcionário Setor Privado,Ensino Médio Incompleto,24,Casado,Negro,1.0,1100,SC,Santa Catarina,Sul,Brasil
4,5,28,Funcionário Setor Privado,Ensino Médio Completo,15,Casado,Negro,0.0,3430,PB,Paraíba,Nordeste,Brasil


In [12]:
def verifica_dados_duplicados(nome_dataframe, dataframe):
    qtd = dataframe.duplicated().sum()

    print(f'O dataframe {nome_dataframe} possui {qtd} dados duplicados')

In [13]:
verifica_dados_duplicados('dados_clinicos', df_dados_clinicos)
verifica_dados_duplicados('dados_pacientes', df_dados_pacientes)
verifica_dados_duplicados('dados_estado_regiao', df_estado_regiao)

    

O dataframe dados_clinicos possui 301 dados duplicados
O dataframe dados_pacientes possui 85 dados duplicados
O dataframe dados_estado_regiao possui 9 dados duplicados


In [14]:
lista_dataframes = [
    ('dados_clinicos', df_dados_clinicos),
    ('dados_pacientes', df_dados_pacientes),
    ('dados_estado_regiao', df_estado_regiao)
]

for nome_dataframe, dataframe in lista_dataframes:
           
    print('--' * 50)
    verifica_dados_duplicados(nome_dataframe, dataframe)
print('--' * 50)

----------------------------------------------------------------------------------------------------
O dataframe dados_clinicos possui 301 dados duplicados
----------------------------------------------------------------------------------------------------
O dataframe dados_pacientes possui 85 dados duplicados
----------------------------------------------------------------------------------------------------
O dataframe dados_estado_regiao possui 9 dados duplicados
----------------------------------------------------------------------------------------------------


In [21]:
df_dados_estado_regiao_sem_duplicatas = df_estado_regiao.drop_duplicates()

In [19]:
print(df_dados_estado_regiao_sem_duplicatas)

    id_estado sigla               estado        regiao    pais
0           1    AC                 Acre         Norte  Brasil
1           2    AL              Alagoas      Nordeste  Brasil
2           3    AP                Amapá         Norte  Brasil
3           4    AM             Amazonas         Norte  Brasil
4           5    BA                Bahia      Nordeste  Brasil
5           6    CE                Ceará      Nordeste  Brasil
6           7    DF     Distrito Federal  Centro-Oeste  Brasil
7           8    ES       Espírito Santo       Sudeste  Brasil
8           9    GO                Goiás  Centro-Oeste  Brasil
9          10    MA             Maranhão      Nordeste  Brasil
10         11    MT          Mato Grosso  Centro-Oeste  Brasil
11         12    MS   Mato Grosso do Sul  Centro-Oeste  Brasil
12         13    MG         Minas Gerais       Sudeste  Brasil
13         14    PA                 Pará         Norte  Brasil
14         15    PB              Paraíba      Nordeste 

In [22]:
verifica_dados_duplicados('dados_estado_regiao', df_estado_regiao)

O dataframe dados_estado_regiao possui 9 dados duplicados
